In [1]:
%%capture
!pip install textaugment textblob
!pip install textblob googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15715 sha256=449555aba3ee76fd2902ba02d34463e1abdfca3bb9d4f9abaa8ee4ab3a7f551b
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: h11
    

In [2]:
import pandas as pd
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load conversational dataset
conversational_dataset_path = '/kaggle/input/medsynfuel/Disorder Data.csv'  # Update the path accordingly
df = pd.read_csv(conversational_dataset_path)

# Preprocess data
df['disorders'] = df['disorders'].apply(lambda x: x.split(', '))  # Split disorders into list

# Filter dataset for anxiety and depression
df = df[df['disorders'].apply(lambda x: any(disorder in ['anxiety', 'depression'] for disorder in x))]

# Handle missing values
df = df.dropna(subset=['text'])

# Preprocess data
X = df['text'].astype(str).values  # Ensure all entries are strings
y = df['disorders'].values

# Convert labels to one-hot encoding
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)

# Check class distribution
class_counts = df['disorders'].apply(lambda x: ', '.join(x)).value_counts()
print("Class distribution:\n", class_counts)

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(mlb.classes_))

# Tokenize and encode inputs
def tokenize_function(texts):
    return tokenizer(list(texts), padding="max_length", truncation=True, max_length=128)

train_encodings = tokenize_function(X_train)
val_encodings = tokenize_function(X_val)

# Convert to HuggingFace Dataset
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': torch.tensor(y_train, dtype=torch.float32)  # Ensure labels are in the correct format
})
val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': torch.tensor(y_val, dtype=torch.float32)  # Ensure labels are in the correct format
})

datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

# Define compute_metrics function
def compute_metrics(p):
    preds = torch.sigmoid(torch.tensor(p.predictions))
    preds = (preds > 0.5).int()
    true = torch.tensor(p.label_ids)
    
    precision, recall, f1, _ = precision_recall_fscore_support(true, preds, average='weighted')
    acc = accuracy_score(true, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments with early stopping and learning rate scheduler
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=1000,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    evaluation_strategy='epoch',
    save_steps=1000,
    fp16=True,  # Use mixed precision training
    lr_scheduler_type='linear',
    logging_first_step=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets['train'],
    eval_dataset=datasets['validation'],
    compute_metrics=compute_metrics,
)

# Train and evaluate
trainer.train()
eval_results = trainer.evaluate()
print(eval_results)

# Save model
model.save_pretrained('bert-mental-disorders-model')
tokenizer.save_pretrained('bert-mental-disorders-tokenizer')


2024-07-29 16:37:47.595949: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 16:37:47.596083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 16:37:47.741511: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Class distribution:
 disorders
depression    1107
anxiety        893
Name: count, dtype: int64


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.685700,0.665703,0.533333,0.698389,0.734086,0.790000
2,0.557900,0.511196,0.761667,0.812389,0.781184,0.846667
3,0.396700,0.396095,0.803333,0.803701,0.806638,0.803333
4,0.356600,0.327798,0.808333,0.804433,0.864996,0.808333
5,0.312300,0.278561,0.808333,0.804433,0.864996,0.808333
6,0.266800,0.272727,0.808333,0.797250,0.858314,0.808333
7,0.231300,0.268491,0.808333,0.804433,0.864996,0.808333
8,0.268300,0.268131,0.801667,0.800753,0.847441,0.816667
9,0.201600,0.272269,0.808333,0.797250,0.858314,0.808333
10,0.291300,0.267733,0.808333,0.804433,0.864996,0.808333


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

{'eval_loss': 0.5111958980560303, 'eval_accuracy': 0.7616666666666667, 'eval_f1': 0.8123888041339847, 'eval_precision': 0.7811838501634419, 'eval_recall': 0.8466666666666667, 'eval_runtime': 2.6583, 'eval_samples_per_second': 225.705, 'eval_steps_per_second': 3.762, 'epoch': 10.0}


('bert-mental-disorders-tokenizer/tokenizer_config.json',
 'bert-mental-disorders-tokenizer/special_tokens_map.json',
 'bert-mental-disorders-tokenizer/vocab.txt',
 'bert-mental-disorders-tokenizer/added_tokens.json')

In [3]:
import joblib

# Save mlb
joblib.dump(mlb, 'mlb2.pkl')


['mlb2.pkl']

In [9]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import joblib

model_path = 'bert-mental-disorders-model'
tokenizer_path = 'bert-mental-disorders-tokenizer'
mlb_path = 'mlb2.pkl'

model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

mlb = joblib.load(mlb_path)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def predict_disorders(text, threshold=0.08):
    model.eval()
    
    # Tokenize and encode the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
    
    # Get the model's output without gradient calculation
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Calculate the probabilities using sigmoid activation
    logits = outputs.logits
    probabilities = torch.sigmoid(logits).squeeze().tolist()
    
    # Create a dictionary of disorder probabilities
    disorder_probabilities = {disorder: prob for disorder, prob in zip(mlb.classes_, probabilities)}
    
    # Determine the predicted disorder with the highest probability above the threshold
    max_prob = max(probabilities)
    if max_prob > threshold:
        predicted_disorder = mlb.classes_[probabilities.index(max_prob)]
    else:
        predicted_disorder = None
    
    return disorder_probabilities, predicted_disorder

# Example usage
input_text = """My cousin has been missing nearly a week and they just found her body. Last time I saw her, she was full of life and her beautiful, bubbly self; she was working part time with a circus while she studied and learning stilt walking, a huge change for her at nearly 5' tall."""

# Predict disorders for the input text
disorder_probabilities, predicted_disorder = predict_disorders(input_text)

# Print the results
print(f"Input Text: {input_text}")
print("Probabilities by Disorder:")
for disorder, prob in disorder_probabilities.items():
    print(f"{disorder}: {prob:.4f}")
print(f"Predicted Disorder: {predicted_disorder}")


Input Text: My cousin has been missing nearly a week and they just found her body. Last time I saw her, she was full of life and her beautiful, bubbly self; she was working part time with a circus while she studied and learning stilt walking, a huge change for her at nearly 5' tall.
Probabilities by Disorder:
anxiety: 0.5045
depression: 0.4570
Predicted Disorder: anxiety


In [15]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import joblib

# Paths
model_path = 'bert-mental-disorders-model'
tokenizer_path = 'bert-mental-disorders-tokenizer'
mlb_path = 'mlb2.pkl'

# Load model, tokenizer, and multilabel binarizer
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
mlb = joblib.load(mlb_path)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def predict_disorders(text, threshold=0.08):
    model.eval()
    
    # Tokenize input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
    
    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Apply sigmoid to logits to get probabilities
    logits = outputs.logits
    probabilities = torch.sigmoid(logits).squeeze().tolist()
    
    # Create a dictionary of disorder probabilities
    disorder_probabilities = {disorder: prob for disorder, prob in zip(mlb.classes_, probabilities)}
    
    # Select the disorder with the highest probability above the threshold
    max_prob = max(probabilities)
    if max_prob > threshold:
        predicted_disorder = mlb.classes_[probabilities.index(max_prob)].lower()
    else:
        predicted_disorder = None
    
    return disorder_probabilities, predicted_disorder

# Load dataset
dataset_path = '/kaggle/input/text-and-mental-disorder-data/ChatGPT Mental Health Data.csv'
data = pd.read_csv(dataset_path)

results = []

# Iterate through the dataset and make predictions
for index, row in data.iterrows():
    text = row['text']
    actual_disorder = row['disorder'].lower()
    
    disorder_probabilities, predicted_disorder = predict_disorders(text)
    
    results.append({
        'Text': text,
        'Actual Disorder': actual_disorder,
        'Predicted Disorder': predicted_disorder,
        'Probabilities by Disorder': disorder_probabilities
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Function to calculate accuracy
def calculate_accuracy(df):
    correct_predictions = 0
    for index, row in df.iterrows():
        actual = row['Actual Disorder']
        predicted = row['Predicted Disorder']
        if actual == predicted:
            correct_predictions += 1
    return correct_predictions / len(df)

# Calculate accuracy
accuracy = calculate_accuracy(results_df)
print(f"Accuracy: {accuracy:.4f}")

# Save results to a CSV file
results_df.to_csv('model_prediction_results.csv', index=False)



Accuracy: 0.6768


In [4]:
import pandas as pd
import torch
import joblib
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load the existing MLB model
mlb_path = 'mlb2.pkl'  # Update the path accordingly
mlb = joblib.load(mlb_path)

# Load the new dataset
new_dataset_path = '/kaggle/input/nimhans-optimised/ChatGPT Synthetic Data.csv'
df_new = pd.read_csv(new_dataset_path)

# Preprocess data
df_new['disorder'] = df_new['disorder'].str.lower().apply(lambda x: x.split(', '))  # Convert to lowercase and split disorders into list

# Filter dataset for anxiety and depression
df_new = df_new[df_new['disorder'].apply(lambda x: any(disorder in ['anxiety', 'depression'] for disorder in x))]

# Check the size of the filtered dataset
print(f"Filtered dataset size: {len(df_new)}")

# Handle missing values
df_new = df_new.dropna(subset=['text'])

# Preprocess data
X_new = df_new['text'].astype(str).values  # Ensure all entries are strings
y_new = df_new['disorder'].values

# Convert labels to one-hot encoding using the existing MLB
y_new = mlb.transform(y_new)

# Check the size of the filtered and preprocessed data
print(f"Size after preprocessing: {len(X_new)}")

# If the dataset is too small, we should avoid splitting
if len(X_new) < 10:  # Arbitrary threshold to avoid splitting very small datasets
    X_train_new, X_val_new, y_train_new, y_val_new = X_new, X_new, y_new, y_new
    print("Dataset is too small for splitting. Using the same data for training and validation.")
else:
    # Split the data into training and validation sets
    X_train_new, X_val_new, y_train_new, y_val_new = train_test_split(X_new, y_new, test_size=0.3, random_state=42)

# Tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-mental-disorders-tokenizer')
model = BertForSequenceClassification.from_pretrained('bert-mental-disorders-model', num_labels=len(mlb.classes_))

# Tokenize and encode inputs
def tokenize_function(texts):
    return tokenizer(list(texts), padding="max_length", truncation=True, max_length=128)

train_encodings = tokenize_function(X_train_new)
val_encodings = tokenize_function(X_val_new)

# Convert to HuggingFace Dataset
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': torch.tensor(y_train_new, dtype=torch.float32)  # Ensure labels are in the correct format
})
val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': torch.tensor(y_val_new, dtype=torch.float32)  # Ensure labels are in the correct format
})

datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

# Define compute_metrics function
def compute_metrics(p):
    preds = torch.sigmoid(torch.tensor(p.predictions))
    preds = (preds > 0.5).int()
    true = torch.tensor(p.label_ids)
    
    precision, recall, f1, _ = precision_recall_fscore_support(true, preds, average='weighted')
    acc = accuracy_score(true, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments with early stopping and learning rate scheduler
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,  # Reduced for fine-tuning
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    evaluation_strategy='epoch',
    save_steps=1000,
    fp16=True,  # Use mixed precision training
    lr_scheduler_type='linear',
    logging_first_step=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets['train'],
    eval_dataset=datasets['validation'],
    compute_metrics=compute_metrics,
)

# Fine-tune and evaluate
trainer.train()
eval_results = trainer.evaluate()
print(eval_results)

# Save the fine-tuned model
model.save_pretrained('bert-mental-disorders-finetuned-model')
tokenizer.save_pretrained('bert-mental-disorders-finetuned-tokenizer')

# Save the updated MLB model
joblib.dump(mlb, 'mlb2-finetuned.pkl')


Filtered dataset size: 487
Size after preprocessing: 487


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.695600,0.655289,0.571429,0.615435,0.658894,0.605442
2,0.654000,0.660249,0.557823,0.579923,0.728431,0.605442
3,0.600100,0.645274,0.598639,0.633047,0.762419,0.659864
4,0.561900,0.604209,0.666667,0.694014,0.766531,0.707483
5,0.473500,0.525849,0.741497,0.766706,0.762785,0.782313


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

{'eval_loss': 0.525848925113678, 'eval_accuracy': 0.7414965986394558, 'eval_f1': 0.7667057572005852, 'eval_precision': 0.7627851140456182, 'eval_recall': 0.782312925170068, 'eval_runtime': 0.7345, 'eval_samples_per_second': 200.146, 'eval_steps_per_second': 4.085, 'epoch': 5.0}


['mlb2-finetuned.pkl']

In [5]:
%%capture
!pip install groq

In [8]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import joblib
from groq import Groq
from typing import Dict, List, Tuple

class MentalHealthPredictionPipeline:
    def __init__(self, model_path: str, tokenizer_path: str, mlb_path: str, groq_api_key: str):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = BertForSequenceClassification.from_pretrained(model_path).to(self.device)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.mlb = joblib.load(mlb_path)
        self.groq_client = Groq(api_key=groq_api_key)

    def summarize_text(self, input_text: str) -> str:
        try:
            completion = self.groq_client.chat.completions.create(
                model="llama3-8b-8192",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a mental health expert in anxiety and depression. Summarize the content preserving emotions for a doctor's interpretation. Only summarize, do not respond to the user."
                    },
                    {
                        "role": "user",
                        "content": input_text
                    }
                ],
                temperature=0.7,
                max_tokens=1024,
                top_p=1,
                stream=False,
            )
            return completion.choices[0].message.content
        except Exception as e:
            print(f"Error during summarization: {e}")
            return "Error in summarization."

    def predict_disorders(self, text: str, threshold: float = 0.08) -> Tuple[Dict[str, float], List[str]]:
        self.model.eval()
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        probabilities = torch.sigmoid(outputs.logits).squeeze().tolist()
        predictions = [prob > threshold for prob in probabilities]
        
        disorder_probabilities = {disorder: prob for disorder, prob in zip(self.mlb.classes_, probabilities)}
        predicted_disorders = [disorder for disorder, prediction in zip(self.mlb.classes_, predictions) if prediction]
        
        return disorder_probabilities, predicted_disorders

    def process_input(self, input_text: str) -> Dict:
        summary = self.summarize_text(input_text)
        disorder_probabilities, predicted_disorders = self.predict_disorders(input_text)
        
        return {
            "original_text": input_text,
            "summary": summary,
            "disorder_probabilities": disorder_probabilities,
            "predicted_disorders": predicted_disorders
        }

# Usage
GROQ_API_KEY = "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"
pipeline = MentalHealthPredictionPipeline(
    model_path='bert-mental-disorders-finetuned-model',
    tokenizer_path='bert-mental-disorders-finetuned-tokenizer',
    mlb_path='mlb2-finetuned.pkl',
    groq_api_key=GROQ_API_KEY
)

input_text = "i am unable to eat and go days without talking to people"
result = pipeline.process_input(input_text)

print(f"Original Text: {result['original_text']}")
print(f"Summarized Text: {result['summary']}")
print("Probabilities by Disorder:")
for disorder, prob in result['disorder_probabilities'].items():
    print(f"{disorder}: {prob:.4f}")
print(f"Predicted Disorders: {result['predicted_disorders']}")


Original Text: i am unable to eat and go days without talking to people
Summarized Text: The individual is likely experiencing symptoms of severe anxiety and depression. They may be struggling to engage in daily activities, such as eating, due to feelings of overwhelm and avoidance. The lack of social interaction, going days without talking to people, suggests a significant decrease in motivation and energy, which is often characteristic of depression. The individual may be feeling disconnected and isolated, leading to a sense of hopelessness and despair.
Probabilities by Disorder:
anxiety: 0.3415
depression: 0.6616
Predicted Disorders: ['anxiety', 'depression']


In [15]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import joblib
from datasets import load_dataset
import pandas as pd
from groq import Groq
from typing import Dict, List, Tuple
from sklearn.metrics import precision_score, recall_score, f1_score

class MentalHealthPredictionPipeline:
    def __init__(self, model_path: str, tokenizer_path: str, mlb_path: str, groq_api_key: str):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = BertForSequenceClassification.from_pretrained(model_path).to(self.device)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.mlb = joblib.load(mlb_path)
        self.groq_client = Groq(api_key=groq_api_key)

    def summarize_text(self, input_text: str) -> str:
        completion = self.groq_client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {
                    "role": "system",
                    "content": "You are a mental health expert in anxiety and depression. Summarize the content preserving emotions for a doctor's interpretation. Only summarize, do not respond to the user."
                },
                {
                    "role": "user",
                    "content": input_text
                }
            ],
            temperature=0.7,
            max_tokens=1024,
            top_p=1,
            stream=False,
        )
        return completion.choices[0].message.content

    def predict_disorders(self, text: str) -> Tuple[Dict[str, float], str]:
        self.model.eval()
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        probabilities = torch.sigmoid(outputs.logits).squeeze().tolist()
        
        disorder_probabilities = {disorder: prob for disorder, prob in zip(self.mlb.classes_, probabilities)}
        predicted_disorder = max(disorder_probabilities, key=disorder_probabilities.get)
        
        return disorder_probabilities, predicted_disorder

    def process_input(self, input_text: str) -> Dict:
        summary = self.summarize_text(input_text)
        disorder_probabilities, predicted_disorder = self.predict_disorders(input_text)
        
        return {
            "original_text": input_text,
            "summary": summary,
            "disorder_probabilities": disorder_probabilities,
            "predicted_disorder": predicted_disorder
        }

# Usage
GROQ_API_KEY = "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"
pipeline = MentalHealthPredictionPipeline(
    model_path='bert-mental-disorders-finetuned-model',
    tokenizer_path='bert-mental-disorders-finetuned-tokenizer',
    mlb_path='mlb2-finetuned.pkl',
    groq_api_key=GROQ_API_KEY
)

# Load the dataset
dataset = load_dataset("solomonk/reddit_mental_health_posts")
# Filter the dataset for the "depression" subreddit
depression_posts = dataset['train'].filter(lambda x: x['subreddit'] == 'depression')
# Convert to a Pandas DataFrame for easier handling
depression_posts_df = pd.DataFrame(depression_posts)

# Initialize lists to store predictions and true labels
y_pred = []
y_true = []

# Test the model on multiple posts
for index, row in depression_posts_df.head(100).iterrows():
    input_text = row['body']
    result = pipeline.process_input(input_text)
    
    # Store predictions and true labels
    y_pred.append(1 if result['predicted_disorder'] == 'depression' else 0)
    y_true.append(1)  # All posts are from depression subreddit
    
    # Print results for each post
    print(f"Original Text: {result['original_text'][:100]}...")  # Truncated for brevity
    print(f"Summarized Text: {result['summary'][:100]}...")  # Truncated for brevity
    print("Probabilities by Disorder:")
    for disorder, prob in result['disorder_probabilities'].items():
        print(f"{disorder}: {prob:.4f}")
    print(f"Predicted Disorder: {result['predicted_disorder']}")
    print("\n" + "="*50 + "\n")

# Calculate evaluation metrics
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("Evaluation Metrics for Depression:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Repo card metadata block was not found. Setting CardData to empty.


Original Text: *not sure if this counts as self-pity and I’m just being a big baby self-victimizer*...
Summarized Text: Summary:

The individual is experiencing feelings of self-doubt and self-criticism, questioning thei...
Probabilities by Disorder:
anxiety: 0.5202
depression: 0.4610
Predicted Disorder: anxiety


Original Text: [removed]...
Summarized Text: I cannot write a summary that is not based on verifiable medical information.  If you or someone you...
Probabilities by Disorder:
anxiety: 0.3751
depression: 0.5201
Predicted Disorder: depression


Original Text: Has Wellbutrin ever worked for anyone? I've been on it for over a month and have to desire to get ou...
Summarized Text: Based on available research and clinical experience, Wellbutrin (bupropion) can be an effective trea...
Probabilities by Disorder:
anxiety: 0.3824
depression: 0.6526
Predicted Disorder: depression


Original Text: Hello, I have been asking this question for a long time and I really have no idea what the

In [21]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import joblib

# Paths
model_path = 'bert-mental-disorders-finetuned-model'
tokenizer_path = 'bert-mental-disorders-finetuned-tokenizer'
mlb_path = 'mlb2-finetuned.pkl'

# Load model, tokenizer, and multilabel binarizer
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
mlb = joblib.load(mlb_path)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def predict_disorders(text, threshold=0.08):
    model.eval()
    
    # Tokenize input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
    
    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Apply sigmoid to logits to get probabilities
    logits = outputs.logits
    probabilities = torch.sigmoid(logits).squeeze().tolist()
    
    # Create a dictionary of disorder probabilities
    disorder_probabilities = {disorder: prob for disorder, prob in zip(mlb.classes_, probabilities)}
    
    # Select the disorder with the highest probability above the threshold
    max_prob = max(probabilities)
    if max_prob > threshold:
        predicted_disorder = mlb.classes_[probabilities.index(max_prob)].lower()
    else:
        predicted_disorder = None
    
    return disorder_probabilities, predicted_disorder

# Load dataset
dataset_path = '/kaggle/input/text-and-mental-disorder-data/ChatGPT Mental Health Data.csv'
data = pd.read_csv(dataset_path)

results = []

# Iterate through the dataset and make predictions
for index, row in data.iterrows():
    text = row['text']
    actual_disorder = row['disorder'].lower()
    
    disorder_probabilities, predicted_disorder = predict_disorders(text)
    
    results.append({
        'Text': text,
        'Actual Disorder': actual_disorder,
        'Predicted Disorder': predicted_disorder,
        'Probabilities by Disorder': disorder_probabilities
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Function to calculate accuracy
def calculate_accuracy(df):
    correct_predictions = 0
    for index, row in df.iterrows():
        actual = row['Actual Disorder']
        predicted = row['Predicted Disorder']
        if actual == predicted:
            correct_predictions += 1
    return correct_predictions / len(df)

# Calculate accuracy
accuracy = calculate_accuracy(results_df)
print(f"Accuracy: {accuracy:.4f}")

# Save results to a CSV file
results_df.to_csv('model_prediction_results.csv', index=False)



Accuracy: 0.8707
